In [1]:
'''

用戶傳照片消息給Line

我方使用 消息的id 向Line 索取照片

'''

'\n\n用戶傳照片消息給Line\n\n我方使用 消息的id 向Line 索取照片\n\n'

In [2]:
"""

啟用伺服器基本樣板

"""
# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 
from linebot.exceptions import (
    InvalidSignatureError
)

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,ImageMessage
)

# 載入設定檔

import json
secretFileContentJson=json.load(open("../line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")


# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "../images/" )

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'


In [3]:
'''

若收到圖片消息時，

先回覆用戶文字消息，並從Line上將照片拿回。

'''
import time
@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='圖片已上傳'+ ' ' + time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime())))
    
    user_profile = line_bot_api.get_profile(event.source.user_id)
    
    message_content = line_bot_api.get_message_content(event.message.id)
    with open('../images/'+ vars(user_profile)["user_id"] + "_" + time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime())+'.jpg', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

In [4]:
# @handler.add(MessageEvent, message=ImageMessage)
# def handle_message(event):
#     user_profile = line_bot_api.get_profile(event.source.user_id)
#     print(vars(user_profile)['user_id'])

In [5]:
'''

啟動Server

'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.3 - - [19/Apr/2019 09:32:36] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [19/Apr/2019 09:32:36] "POST / HTTP/1.1" 200 -
172.17.0.3 - - [19/Apr/2019 09:32:46] "POST / HTTP/1.1" 200 -
